In [2]:
class Hello(object):
    def hello(self,name='world'):
        print('hello,%s.' %name)
        
h=Hello()
h.hello()

hello,world.


In [3]:
print(type(Hello))

<class 'type'>


In [4]:
print(type(h))

<class '__main__.Hello'>


In [5]:
def fn(self,name='world'):
    print('hello,%s.' %name)
    
Hello = type('hello',(object,),dict(hello=fn))
h=Hello()
h.hello()

hello,world.


In [6]:
print(type(Hello))

<class 'type'>


In [7]:
print(type(h))

<class '__main__.hello'>


In [9]:
class ListMetaclass(type):
    def __new__(cls,name,bases,attrs):
        attrs['add'] = lambda self,value:self.append(value)
        return type.__new__(cls,name,bases,attrs)
    
class MyList(list,metaclass=ListMetaclass):
    pass

L = MyList()
L.add(1)
L

[1]

In [16]:

#u = User(id=12345,name='Michael',email='test@orm.org',password='my-pwd')
#u.save()

class Field(object):
    
    def __init__(self,name,column_type):
        self.name=name
        self.column_type=column_type
        
    def __str__(self):
        return '<%s:%s>' % (self.__class__.__name__,self.name)
    
class StringField(Field):
    
    def __init__(self,name):
        super(StringField,self).__init__(name,'varchar(100)')
        
class IntegerField(Field):
    
    def __init__(self,name):
        super(IntegerField,self).__init__(name,'bigint')
        
class ModelMetaclass(type):
    
    def __new__(cls,name,bases,attrs):
        if name=='Model':
            return type.__new__(cls,name,bases,attrs)
        print('Found model: %s' % name)
        mappings = dict()
        for k,v in attrs.items():
            if isinstance(v,Field):
                print('Found mapping : %s ==> %s' % (k,v))
                mappings[k]=v
        for k in mappings.keys():
            attrs.pop(k)
        attrs['__mappings__']=mappings
        attrs['__table__']=name
        return type.__new__(cls,name,bases,attrs)
    
    
class Model(dict,metaclass=ModelMetaclass):
    
    def __init__(self,**kw):
        super(Model, self).__init__(**kw)
        
    def __getattr__(self,key):
        try:
            return self[key]
        except KeyError:
            raise AttributeError(r"'Model' object has no attribute '%s'" % key)
            
    def __setattr__(self,key,value):
        self[key] = value
        
    def save(self):
        fields = []
        params = []
        args = []
        for k, v in self.__mappings__.items():
            fields.append(v.name)
            params.append('?')
            args.append(getattr(self, k, None))
        sql = 'insert into %s (%s) values (%s)' % (self.__table__, ','.join(fields), ','.join(params))
        print('SQL: %s' % sql)
        print('ARGS: %s' % str(args))
        
class User(Model):
    id = IntegerField('id')
    name = StringField('username')
    email = StringField('email')
    password = StringField('password')
    
u = User(id=12345,name='Michael',email='test@orm.org',password='my-pwd')
u.save()


Found model: User
Found mapping : id ==> <IntegerField:id>
Found mapping : name ==> <StringField:username>
Found mapping : email ==> <StringField:email>
Found mapping : password ==> <StringField:password>
SQL: insert into User (id,username,email,password) values (?,?,?,?)
ARGS: [12345, 'Michael', 'test@orm.org', 'my-pwd']
